In [15]:
import numpy as np
import pandas as pd
import compiler_gym
from compiler_gym.wrappers import RuntimePointEstimateReward

from compopt.constants import RUNNABLE_BMS

In [38]:
from compopt.optimization_target import OptimizationTarget

In [49]:
target = OptimizationTarget(OptimizationTarget.RUNTIME)

In [2]:
estimator = np.median

In [61]:
list(env.datasets['benchmark://chstone-v0'].benchmark_uris())

['benchmark://chstone-v0/adpcm',
 'benchmark://chstone-v0/aes',
 'benchmark://chstone-v0/blowfish',
 'benchmark://chstone-v0/dfadd',
 'benchmark://chstone-v0/dfdiv',
 'benchmark://chstone-v0/dfmul',
 'benchmark://chstone-v0/dfsin',
 'benchmark://chstone-v0/gsm',
 'benchmark://chstone-v0/jpeg',
 'benchmark://chstone-v0/mips',
 'benchmark://chstone-v0/motion',
 'benchmark://chstone-v0/sha']

In [3]:
env = compiler_gym.make(
    'llvm-v0',
    benchmark='benchmark://cbench-v1/susan',
    observation_space='Autophase'
)

In [4]:
bms = list(env.datasets['benchmark://cbench-v1'].benchmark_uris())
for bm in bms:
    env = compiler_gym.make(
        'llvm-v0',
        benchmark=bm,
        observation_space='Programl'
    )
    obs = env.reset()
    n = obs.number_of_nodes()
    e = obs.number_of_edges()
    print(bm, 'node size:', n, 'edge size:', e)

benchmark://cbench-v1/adpcm node size: 1184 edge size: 2105
benchmark://cbench-v1/bitcount node size: 1826 edge size: 3146
benchmark://cbench-v1/blowfish node size: 8684 edge size: 14976
benchmark://cbench-v1/bzip2 node size: 59412 edge size: 105845
benchmark://cbench-v1/crc32 node size: 512 edge size: 907
benchmark://cbench-v1/dijkstra node size: 891 edge size: 1593


KeyboardInterrupt: 

In [5]:
def make_env(bm):
    env = compiler_gym.make(
        'llvm-v0',
        benchmark=bm,
        observation_space='Autophase'
    )
    env = RuntimePointEstimateReward(env)

    return env

In [6]:
def runtime(env):
    env.unwrapped.observation["Runtime"]  # warmup
    runtimes_o3 = env.unwrapped.observation["Runtime"].tolist()
    
    return estimator(runtimes_o3)

In [62]:
bms = ['benchmark://chstone-v0/adpcm',
 'benchmark://chstone-v0/aes',
 'benchmark://chstone-v0/blowfish',
 'benchmark://chstone-v0/dfadd',
 'benchmark://chstone-v0/dfdiv',
 'benchmark://chstone-v0/dfmul',
 'benchmark://chstone-v0/dfsin',
 'benchmark://chstone-v0/gsm',
 'benchmark://chstone-v0/jpeg',
 'benchmark://chstone-v0/mips',
 'benchmark://chstone-v0/motion',
 'benchmark://chstone-v0/sha']

In [65]:
res = {}
o3_res = {}
speedups = []
for bm in RUNNABLE_BMS:
    env = make_env(bm)
    speedup = target.final_reward(env)
    print(speedup)
    speedups.append(speedup)
#     env.reset()
#     res[bm] = runtime(env)
    
#     env = make_env(bm)
    
#     env.reset()
#     env.send_param("llvm.apply_baseline_optimizations", "-O3")
#     o3_res[bm] = runtime(env)

0.7510148396201433
0.9832047986289632
2.1084607393690296
1.2438732815301852
1.006625347296431
0.47261352453254596
0.9041830116130035
1.0180649361217349
0.9583333333333334
0.8600723595066143
1.0080596413459602
1.0102554179566563
0.7291480663502654
0.6839989815218974
1.1304347826086958
1.3347248576850095
1.4230430958663147
0.6778597927104448


In [58]:
(np.array(speedups) > 1).sum()

9

In [10]:
res

{'benchmark://cbench-v1/bitcount': 0.151849,
 'benchmark://cbench-v1/blowfish': 0.0035884999999999997,
 'benchmark://cbench-v1/bzip2': 0.097198,
 'benchmark://cbench-v1/crc32': 0.0038705,
 'benchmark://cbench-v1/dijkstra': 0.0023225,
 'benchmark://cbench-v1/gsm': 0.035087999999999994,
 'benchmark://cbench-v1/jpeg-c': 0.018074,
 'benchmark://cbench-v1/jpeg-d': 0.007137,
 'benchmark://cbench-v1/patricia': 0.0024574999999999996,
 'benchmark://cbench-v1/qsort': 0.042591000000000004,
 'benchmark://cbench-v1/sha': 0.0028855,
 'benchmark://cbench-v1/stringsearch': 0.002953,
 'benchmark://cbench-v1/stringsearch2': 0.0105525,
 'benchmark://cbench-v1/susan': 0.025582,
 'benchmark://cbench-v1/tiff2bw': 0.0089245,
 'benchmark://cbench-v1/tiff2rgba': 0.0105755,
 'benchmark://cbench-v1/tiffdither': 0.0077925,
 'benchmark://cbench-v1/tiffmedian': 0.013024500000000001}

In [11]:
o3_res

{'benchmark://cbench-v1/bitcount': 0.12199650000000001,
 'benchmark://cbench-v1/blowfish': 0.0038335,
 'benchmark://cbench-v1/bzip2': 0.1818715,
 'benchmark://cbench-v1/crc32': 0.004181000000000001,
 'benchmark://cbench-v1/dijkstra': 0.0026100000000000003,
 'benchmark://cbench-v1/gsm': 0.0244225,
 'benchmark://cbench-v1/jpeg-c': 0.017019,
 'benchmark://cbench-v1/jpeg-d': 0.0085135,
 'benchmark://cbench-v1/patricia': 0.0029365,
 'benchmark://cbench-v1/qsort': 0.031739500000000004,
 'benchmark://cbench-v1/sha': 0.0025325,
 'benchmark://cbench-v1/stringsearch': 0.002928,
 'benchmark://cbench-v1/stringsearch2': 0.007665999999999999,
 'benchmark://cbench-v1/susan': 0.019978,
 'benchmark://cbench-v1/tiff2bw': 0.010589000000000001,
 'benchmark://cbench-v1/tiff2rgba': 0.016427999999999998,
 'benchmark://cbench-v1/tiffdither': 0.008409,
 'benchmark://cbench-v1/tiffmedian': 0.012163}

In [13]:
res

{'benchmark://cbench-v1/bitcount': 0.149324,
 'benchmark://cbench-v1/blowfish': 0.0037015,
 'benchmark://cbench-v1/bzip2': 0.09635650000000001,
 'benchmark://cbench-v1/crc32': 0.0034289999999999998,
 'benchmark://cbench-v1/dijkstra': 0.002533,
 'benchmark://cbench-v1/gsm': 0.036967,
 'benchmark://cbench-v1/jpeg-c': 0.0176065,
 'benchmark://cbench-v1/jpeg-d': 0.008311,
 'benchmark://cbench-v1/patricia': 0.0026905,
 'benchmark://cbench-v1/qsort': 0.032676,
 'benchmark://cbench-v1/sha': 0.0027595,
 'benchmark://cbench-v1/stringsearch': 0.0025814999999999996,
 'benchmark://cbench-v1/stringsearch2': 0.0090225,
 'benchmark://cbench-v1/susan': 0.030331999999999998,
 'benchmark://cbench-v1/tiff2bw': 0.009843500000000002,
 'benchmark://cbench-v1/tiff2rgba': 0.012143000000000001,
 'benchmark://cbench-v1/tiffdither': 0.0087895,
 'benchmark://cbench-v1/tiffmedian': 0.011491}

In [14]:
o3_res

{'benchmark://cbench-v1/bitcount': 0.1208835,
 'benchmark://cbench-v1/blowfish': 0.0037194999999999997,
 'benchmark://cbench-v1/bzip2': 0.19269950000000002,
 'benchmark://cbench-v1/crc32': 0.0034159999999999998,
 'benchmark://cbench-v1/dijkstra': 0.00297,
 'benchmark://cbench-v1/gsm': 0.023918500000000002,
 'benchmark://cbench-v1/jpeg-c': 0.015954000000000003,
 'benchmark://cbench-v1/jpeg-d': 0.006951,
 'benchmark://cbench-v1/patricia': 0.0027254999999999996,
 'benchmark://cbench-v1/qsort': 0.031465,
 'benchmark://cbench-v1/sha': 0.002719,
 'benchmark://cbench-v1/stringsearch': 0.003478,
 'benchmark://cbench-v1/stringsearch2': 0.0066175,
 'benchmark://cbench-v1/susan': 0.0198905,
 'benchmark://cbench-v1/tiff2bw': 0.011373000000000001,
 'benchmark://cbench-v1/tiff2rgba': 0.0169615,
 'benchmark://cbench-v1/tiffdither': 0.009925,
 'benchmark://cbench-v1/tiffmedian': 0.012907}

In [19]:
df = pd.DataFrame([res, o3_res]).T

In [20]:
df

,0,1
benchmark://cbench-v1/bitcount,0.149324,0.120884
benchmark://cbench-v1/blowfish,0.003701,0.003719
benchmark://cbench-v1/bzip2,0.096357,0.192700
benchmark://cbench-v1/crc32,0.003429,0.003416
benchmark://cbench-v1/dijkstra,0.002533,0.002970
benchmark://cbench-v1/gsm,0.036967,0.023919
benchmark://cbench-v1/jpeg-c,0.017607,0.015954
benchmark://cbench-v1/jpeg-d,0.008311,0.006951
benchmark://cbench-v1/patricia,0.002691,0.002725
benchmark://cbench-v1/qsort,0.032676,0.031465


In [23]:
(df[0] > df[1]).sum()

9

In [32]:
df = pd.DataFrame([res, o3_res]).T

In [33]:
df

,0,1
benchmark://cbench-v1/bitcount,0.151785,0.149651
benchmark://cbench-v1/blowfish,0.003610,0.003763
benchmark://cbench-v1/bzip2,0.100328,0.099518
benchmark://cbench-v1/crc32,0.004895,0.004829
benchmark://cbench-v1/dijkstra,0.003322,0.003111
benchmark://cbench-v1/gsm,0.035873,0.036157
benchmark://cbench-v1/jpeg-c,0.017641,0.018528
benchmark://cbench-v1/jpeg-d,0.008636,0.008634
benchmark://cbench-v1/patricia,0.003569,0.003567
benchmark://cbench-v1/qsort,0.042244,0.043707


In [27]:
df

,0,1
benchmark://cbench-v1/bitcount,0.151785,0.149651
benchmark://cbench-v1/blowfish,0.003610,0.003763
benchmark://cbench-v1/bzip2,0.100328,0.099518
benchmark://cbench-v1/crc32,0.004895,0.004829
benchmark://cbench-v1/dijkstra,0.003322,0.003111
benchmark://cbench-v1/gsm,0.035873,0.036157
benchmark://cbench-v1/jpeg-c,0.017641,0.018528
benchmark://cbench-v1/jpeg-d,0.008636,0.008634
benchmark://cbench-v1/patricia,0.003569,0.003567
benchmark://cbench-v1/qsort,0.042244,0.043707


In [34]:
(df[0] > df[1])

benchmark://cbench-v1/bitcount          True
benchmark://cbench-v1/blowfish         False
benchmark://cbench-v1/bzip2             True
benchmark://cbench-v1/crc32             True
benchmark://cbench-v1/dijkstra          True
benchmark://cbench-v1/gsm              False
benchmark://cbench-v1/jpeg-c           False
benchmark://cbench-v1/jpeg-d            True
benchmark://cbench-v1/patricia          True
benchmark://cbench-v1/qsort            False
benchmark://cbench-v1/sha               True
benchmark://cbench-v1/stringsearch     False
benchmark://cbench-v1/stringsearch2    False
benchmark://cbench-v1/susan             True
benchmark://cbench-v1/tiff2bw          False
benchmark://cbench-v1/tiff2rgba        False
benchmark://cbench-v1/tiffdither        True
benchmark://cbench-v1/tiffmedian       False
dtype: bool